In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:10:24 of 0:10:24 remaining]
Chain 1:  10% [0:00:52 of 0:00:58 remaining]
Chain 1:  20% [0:00:54 of 0:01:08 remaining]
Chain 1:  30% [0:00:49 of 0:01:10 remaining]
Chain 1:  40% [0:00:43 of 0:01:11 remaining]
Chain 1:  50% [0:00:35 of 0:01:11 remaining]
Chain 1:  60% [0:00:28 of 0:01:11 remaining]
Chain 1:  70% [0:00:21 of 0:01:10 remaining]
Chain 1:  80% [0:00:14 of 0:01:10 remaining]
Chain 1:  90% [0:00:07 of 0:01:09 remaining]
Chain 1: 100% [0:00:00 of 0:01:10 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:44 of 0:00:49 remaining]
Chain 2:  20% [0:00:43 of 0:00:53 remaining]
Chain 2:  30% [0:00:41 of 0:00:58 remaining]
Chain 2:  40% [0:00:37 of 0:01:02 remaining]
Chain 2:  50% [0:00:32 of 0:01:03 remaining]
Chain 2:  60% [0:00:26 of 0:01:05 remaining]
Chain 2:  70% [0:00:20 of 0:01:07 remaining]
Chain 2:  80% [0:00:13 of 0:01:06 remaining]
Chain 2:  90% [0:00:07 of 0:01:07 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[24.6536 113.703 0.899879; 24.3091 113.703 0.899879; … ; 21.8518 114.476 0.894568; 21.8626 113.312 0.918355]

[28.313 112.372 0.943347; 26.91 112.372 0.943347; … ; 26.244 111.65 0.936684; 25.9793 114.459 0.875192]

[26.761 113.619 0.906142; 25.805 113.619 0.906142; … ; 24.5815 114.773 0.898055; 25.0471 114.779 0.87437]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE        MCSE        ESS   
     s2  26.1426029 1.996271324 0.01214892039 0.01988938055 9000.0000
beta[1] 113.8530276 1.851207034 0.01126608724 0.03378534650 3002.2941
beta[2]   0.9056516 0.040723852 0.00024783747 0.00073274534 3088.8126

Quantiles:
            2.5%        25.0%       50.0%       75.0%        97.5%   
     s2  22.4962645  24.75143153  26.0483633  27.4393635  30.27563774
beta[1] 110.2752460 112.56913061 113.8550417 115.0660785 117.53802151
beta[2]   0.8242732   0.87889445   0.9055597   0.9334813   0.98368612



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*